In [4]:
import requests
import pandas as pd

# Step 1: API endpoint
url = "https://api.coingecko.com/api/v3/coins/markets"

# Step 2: Parameters (USD me top 10 coins)
params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 10,
    "page": 1,
    "sparkline": False
}

# Step 3: API se request bhejna
response = requests.get(url, params=params)

# Step 4: JSON response ko Pandas DataFrame me convert karna
data = response.json()
df = pd.DataFrame(data)

# Step 5: Sirf important columns select karna
df = df[["id", "symbol", "current_price", "market_cap", "total_volume", "price_change_percentage_24h"]]

print(df)


             id symbol  current_price     market_cap  total_volume  \
0       bitcoin    btc  113921.000000  2269142879787   56243318977   
1      ethereum    eth    4162.510000   502273435150   33597066611   
2        tether   usdt       1.000000   174451445186  107149225710   
3        ripple    xrp       2.890000   172951798140    4643757437   
4   binancecoin    bnb    1007.310000   140501680607    1938253160   
5        solana    sol     210.460000   114567013832    6745066942   
6      usd-coin   usdc       0.999706    73430886695    8493574436   
7  staked-ether  steth    4162.880000    35540211527      29895954   
8      dogecoin   doge       0.233971    35399211026    2213967983   
9          tron    trx       0.336089    31813468510     508063163   

   price_change_percentage_24h  
0                      3.40796  
1                      3.26445  
2                      0.02250  
3                      2.79188  
4                      3.03829  
5                      3.23385 

# 🔹 Step 1: Basic Analysis (Pandas me)

Ye aapko data ko explore karna sikhaega:

1. Top 5 coins by Market Cap

Market cap sabse zyada kis coin ka hai.

2. Top 5 coins by 24h Percentage Growth

Kaunse coins ne sabse zyada growth ki ek din me.

3. Top 5 coins by Trading Volume

Sabse zyada trade hone wale coins

In [ ]:
#1. Top 5 coins by Market Cap
martek_cap_value = df["market_cap"].sort_values(ascending=False).head(5)
print(martek_cap_value)


0    2269142879787
1     502273435150
2     174451445186
3     172951798140
4     140501680607
Name: market_cap, dtype: int64


In [22]:
print(df.columns)
#2. Top 5 coins by 24h Percentage Growth
coin_Per_growth = df.nlargest(5,"price_change_percentage_24h")[["id","market_cap","price_change_percentage_24h"]]
print(coin_Per_growth)

Index(['id', 'symbol', 'current_price', 'market_cap', 'total_volume',
       'price_change_percentage_24h'],
      dtype='object')
             id     market_cap  price_change_percentage_24h
0       bitcoin  2269142879787                      3.40796
1      ethereum   502273435150                      3.26445
5        solana   114567013832                      3.23385
7  staked-ether    35540211527                      3.19363
4   binancecoin   140501680607                      3.03829


In [21]:
#3. Top 5 coins by Trading Volume
trading_coin = df.nlargest(5,"total_volume")[["id","total_volume"]]
print(trading_coin)


         id  total_volume
2    tether  107149225710
0   bitcoin   56243318977
1  ethereum   33597066611
6  usd-coin    8493574436
5    solana    6745066942
